In [2]:
import pandas as pd
import json

In [2]:
properties_file = open('../hazard_mitigation_properties.json', 'r')
projects_file = open('../hazard_mitigation_projects.json', 'r')
properties_json = json.load(properties_file)
projects_json = json.load(projects_file)

df_properties = pd.DataFrame(properties_json['HazardMitigationAssistanceMitigatedProperties'])
df_projects = pd.DataFrame(projects_json['HazardMitigationAssistanceProjects'])

In [14]:
df_properties.head()

,projectIdentifier,programArea,programFy,disasterNumber,propertyAction,structureType,typeOfResidency,foundationType,county,city,state,stateNumberCode,region,zip,damageCategory,actualAmountPaid,numberOfProperties,id,disaster based
0,DR-4022-0031-R,HMGP,2011,4022.0,Acquisition/Demolition,Non-residential - Private,Other (Specify in Comments),Slab on Grade,Windham,WILMINGTON,Vermont,50,1.0,05363,50 - 99%,NaN,1,23533667-6a8a-4779-b28f-5c4d825ef426,True
1,DR-1976-0020-R,HMGP,2011,1976.0,Acquisition/Demolition,Single Family,Rental,Basement,Jefferson,LOUISVILLE,Kentucky,21,4.0,40211,N/A,63999.0,1,197d75e1-34c8-436a-ba1e-3adc895ca1f8,True
2,DR-1971-0209-R,HMGP,2011,1971.0,Safe Room/Wind Shelter,Single Family,Owner Occupied - Principal Residence,Crawl Space,Jefferson,CONCORD,Alabama,1,4.0,35203,N/A,4000.0,1,44546cd7-20f7-4dd4-ad00-7baf4b8d750f,True
3,DR-1971-0153-R,HMGP,2011,1971.0,Safe Room/Wind Shelter,Single Family,Owner Occupied - Principal Residence,Slab on Grade,Marshall,GUNTERSVILLE,Alabama,1,4.0,35976,N/A,4900.0,1,f3a0ceed-f9e4-4ccb-9a5f-e13595355823,True
4,DR-1917-0013-R,HMGP,2010,1917.0,Safe Room/Wind Shelter,Single Family,Owner Occupied - Principal Residence,Slab on Grade,Canadian,YUKON,Oklahoma,40,6.0,73099,N/A,2000.0,20,de1dc299-3c03-48c9-b36f-2eb45645d000,True


In [21]:
df_projects.head()

,projectIdentifier,programArea,programFy,region,state,stateNumberCode,county,countyCode,disasterNumber,projectCounties,...,subrecipientAdminCostAmt,srmcObligatedAmt,recipientAdminCostAmt,costSharePercentage,benefitCostRatio,netValueBenefits,numberOfFinalProperties,numberOfProperties,id,disaster based
0,DR-1961-0002-R,HMGP,2011,7,Missouri,29,Moniteau,135,1961.0,MONITEAU,...,0.0,0.0,0.0,0.749996,2.175,500586.0,1,1,d9c99d55-5a60-40ea-a758-3a15de021115,True
1,DR-1751-0018-P,HMGP,2008,6,Arkansas,5,Statewide,0,1751.0,STATEWIDE,...,0.0,0.0,0.0,0.750000,0.000,0.0,0,0,1627e572-9f79-4055-9ede-40cf3226c5f6,True
2,DR-1840-0002-R,HMGP,2009,4,Florida,12,Volusia,127,1840.0,VOLUSIA,...,0.0,0.0,0.0,0.750000,1.307,516759.0,0,0,08ea52a7-5126-45a1-aedb-5a5222fbc17c,True
3,DR-1545-0065-R,HMGP,2004,4,Florida,12,Volusia,127,1545.0,VOLUSIA,...,0.0,0.0,0.0,0.750000,69.041,930678.0,0,0,512a4ab1-7ca1-45a3-8390-92b9ef26a07f,True
4,DR-4030-0016-R,HMGP,2011,3,Pennsylvania,42,Columbia,37,4030.0,COLUMBIA,...,0.0,0.0,0.0,0.750000,0.000,0.0,5,6,84e82525-63c5-48b9-8cb6-9e8528171081,True


In [22]:
# check if the property and project are because of a disaster or not, more info on 
# https://www.fema.gov/openfema-data-page/hazard-mitigation-assistance-mitigated-properties-v3
# https://www.fema.gov/openfema-data-page/hazard-mitigation-assistance-projects-v3
df_properties['disaster based'] = df_properties.apply(lambda row: 'DR' in row['projectIdentifier'], axis=1)
df_projects['disaster based'] = df_projects.apply(lambda row: 'DR' in row['projectIdentifier'], axis=1)

In [5]:
df_properties.to_json('../preprocessed-fema-properties.json')
df_projects.to_json('../preprocessed-fema-projects.json')

In [25]:
states = df_properties.groupby(['state', 'programFy', 'programArea'], as_index=False).sum(numeric_only=True)
states.head()

,state,programFy,programArea,disasterNumber,region,actualAmountPaid,numberOfProperties,disaster based
0,Alabama,1990,HMGP,2583.0,12.0,0.0,11,3
1,Alabama,1994,HMGP,16442.0,64.0,0.0,97,16
2,Alabama,1995,HMGP,4188.0,16.0,0.0,5,4
3,Alabama,1996,HMGP,55016.0,204.0,0.0,343,51
4,Alabama,1997,FMA,0.0,12.0,0.0,3,0


In [26]:
df_disasters = pd.read_csv('../Preprocessed-Natural-Disasters.csv', delimiter=';')
df_disasters_us = df_disasters[df_disasters['ISO'] == 'USA']

In [3]:
df_properties = pd.read_json('../preprocessed-fema-properties.json')